

<p style="margin-right:8%; margin-left:8%; font-size:25px; font-weight: 200; line-height: 1.5em">
**Second Part: How to carry out a selection criteria in the Analysis?**

## Example 2: Candidates selection
This example is divided in two parts: 

* Apply a selection criteria 
* Evaluate the distribution performance

In this second step, you must look at **Analyzer_Selection.py** script. This code, again, reads each event of the tree, but instead of filling directly the histograms, it makes a selection for each particle in the event. The selection criteria has been stablished in **Cuts.py** script. Take a look into Analyzer_Selection.py and Cuts.py and notice that a few amount of muons pass the cuts depending on the value of their variables. 

The result of this analysis is a set of histograms that contain valuable statistic information about each kinematic variable for all selected particles produced a long of all events. The aim of this exercise is getting the right muon collection produced in Z->mumu decays. 


**NOTE:** 

For each exercise, to run the code is necessary to import **ROOT**.

In [1]:
# Import ROOT
import ROOT

Welcome to ROOTaaS 6.06/00


In [2]:
cd /home/jupyter/CmsOpenData/Analyzer_Package

/home/jupyter/CmsOpenData/Analyzer_Package


* ### Run the Analysis with the selection criteria, already, stablished in the Cuts module. 
<p style="margin-right:0%; margin-left:0%; font-size:14px; font-weight: 200; line-height: 1.2em">
Before performing the analysis, you must define the cut parameter for all the muon variables, in **Cuts.py** script. Then, you just run the module **Analyzer_Selection.py** and  obtain a root file: **goodHistos.root** stored in **datafiles** folder that contains the histograms for all muons. Each histogram is saved with a name started by *g_* followed by the *name of the kinematic variable*.
<p style="margin-right:0%; margin-left:0%; font-size:14px; font-weight: 200; line-height: 1.2em">
*Cuts (pt_min, eta_max,distance, dB_max, isolation, mass_min, chi2, numValidHits) * 

* #### Calculate the mass 
<p style="margin-right:0%; margin-left:0%; font-size:14px; font-weight: 200; line-height: 1.2em">
for a pair of muons with different charge in the same event.
<p style="margin-right:0%; margin-left:0%; font-size:14px; font-weight: 200; line-height: 1.2em">
mass = $ $ 

def calculate_mass (self, muon1, muon2):
    return mass

In [3]:
# Import the Analyzer module that only performs the reading of selected events from the tree
from Analyzer_Selection import Analyzer

# Call to the analysis and create the root file for histograms
analysis=Analyzer()
analysis.process()

Selected histograms saved in datafiles/goodHistos.root file


* ### Check your g_histogram
<p style="margin-right:0%; margin-left:0%; font-size:14px; font-weight: 200; line-height: 1.2em">
Open the goodHistos.root file for reading and printing the list of histograms saved in it. 
In the selection was added a new variable called efficiency. The *efficiency* is defined by:
<p style="margin-right:0%; margin-left:0%; font-size:14px; font-weight: 200; line-height: 1.2em">
**efficiency = Number of events after cut N / Number of events after cut N-1**

In [4]:
# Get the root file that contains the histograms for selected muons: goodHistos.root.
Gfile=ROOT.TFile("datafiles/goodHistos.root", "read")
# List the histograms in the file
Gfile.ls()

TFile**		datafiles/goodHistos.root	
 TFile*		datafiles/goodHistos.root	
  KEY: TH1F	g_pt;1	Muons Transverse Momentun
  KEY: TH1F	g_px;1	Muons x- Momentun
  KEY: TH1F	g_py;1	Muons y- Momentun
  KEY: TH1F	g_pz;1	Muons z- Momentun
  KEY: TH1F	g_eta;1	Angle Transvese
  KEY: TH1F	g_energy;1	Muons Energy
  KEY: TH1F	g_distance;1	Distance from Primary vertex Z 
  KEY: TH1F	g_charge;1	Muons Charge
  KEY: TH1F	g_normChi2;1	Muons Chi2
  KEY: TH1F	g_numberOfValidHits;1	Number of Valid Hits
  KEY: TH1F	g_dB;1	Impact Parameter
  KEY: TH1F	g_isolation_sumPt;1	IsolationX
  KEY: TH1F	g_isolation_emEt;1	IsolationX
  KEY: TH1F	g_isolation_hadEt;1	IsolationX
  KEY: TH1F	g_mass;1	Inv_mass
  KEY: TH1F	g_efficiency;1	efficiency


* ### Draw Histos
<p style="margin-right:0%; margin-left:0%; font-size:14px; font-weight: 200; line-height: 1.2em">
Once you have that file,  the next step is to plot the histogram. Follow the methods in example1 to plot only one histogram and see below how to display two histograms in the same canvas. 

** 4.** Draw more than one histogram in the same canvas

It can be nice to compare more than one histogram in the same canvas. To do that you must create the two histograms as you want to draw, but just one canvas. For example, we are going to draw histograms for pt variable using the selected and all values.

In [5]:
### Get the root file that contains the histograms for all muons: histos.root.
Hfile=ROOT.TFile("datafiles/histos.root", "read")
Gfile=ROOT.TFile("datafiles/goodHistos.root", "read")

### You must create a variable to store the histograms selected.
histo1=Hfile.Get('h_pt')
histo2=Hfile.Get('g_pt')

### Then create again a new the canvas where the histograms are going to be drawn
canvas = ROOT.TCanvas("myCanvas","All muons: Pt",800,600)

from ROOT import gStyle
### One more time, draw the histogram
histo1.SetTitle("pt good/all Comparation")

# To not print the top-right box of the first histogram
gStyle.SetOptStat(0)
# Draw the histograms in the same canvas
histo1.Draw()
histo2.Draw("same")


### At last, draw the canvas
canvas.Draw()

ReferenceError: attempt to access a null-pointer

** 4.1.** And set again your preferences. ** PAY ATTENTION :** some settings are applied on the histo variable some others on the canvas.  

In [ ]:
### Change the line color 
histo1.SetLineColor(4)
histo2.SetLineColor(6)

### Change the bounds of the histograms for X Axis 
histo1.GetXaxis().SetRangeUser(40, 120);
histo2.GetXaxis().SetRangeUser(0, 200);

### Change the bins for the histograms by diving by a divisor of the initial number of bins. 
## Note: To restore the binning you must to draw the histo again.
histo1.Rebin(2)

### Create the legend. TLegend(x1, y1, x2, y2)
legend =ROOT.TLegend(0.1,0.2,0.30,0.3);
legend.SetHeader("All muons");
legend.AddEntry(histo1, "All pt","l");
legend.AddEntry(histo2, "Selected pt","l");

#legend->AddEntry("gr","Graph with error bars","lep");
legend.Draw();

### Twist linear scale for Y axe to the logaritmic one with the function SetLogy.
canvas.SetLogy()

### Draw the histogram again and check your changes 
canvas.Draw()

In [ ]:
from Histos import Histos

histo = Histos()

histo.DrawHi(('h_pt', 'g_pt'))

In [ ]:
histo.Setting('pt_tunning', ('h_pt','g_pt'), scale = 'LogY', nbin = 2, xlimits = (0,100), legend = True )